In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


from tqdm import tqdm
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vjtitask/sample_submission.csv
/kaggle/input/vjtitask/train.csv
/kaggle/input/vjtitask/test.csv


In [ ]:
!pip install -q pyicu
!pip install -q pycld2
!pip install -q polyglot
!pip install -q textstat
!pip install -q googletrans


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.9/263.9 kB 5.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import gc
import re

import textstat
from scipy import stats
from colorama import Fore, Back, Style, init

import math
import numpy as np
import scipy as sp
import pandas as pd

import random
import networkx as nx
from pandas import Timestamp

from PIL import Image
from IPython.display import SVG
from keras.utils import model_to_dot

import requests
from IPython.display import HTML

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
import matplotlib.pyplot as plt

tqdm.pandas()

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import transformers
import tensorflow as tf

from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from tensorflow.keras.models import Model
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.optimizers import Adam
from tokenizers import BertWordPieceTokenizer
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding
from tensorflow.keras.layers import LSTM, GRU, Conv1D, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.constraints import *
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *

from sklearn import metrics
from sklearn.utils import shuffle
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer,\
                                            CountVectorizer,\
                                            HashingVectorizer

from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer  

import nltk
from textblob import TextBlob

from nltk.corpus import wordnet
from nltk.corpus import stopwords
from googletrans import Translator
from nltk import WordNetLemmatizer
from polyglot.detect import Detector
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer

stopword=set(STOPWORDS)

lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

np.random.seed(0)


In [ ]:
!pip install langdetect 
!pip install indic_transliteration

In [ ]:
from langdetect import detect
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
from collections import Counter

# EDA! 
**visualizing and analysing the comments**

In [2]:
df=pd.read_csv("/kaggle/input/vjtitask/train.csv")
df2=pd.read_csv("/kaggle/input/vjtitask/test.csv")
submission=pd.read_csv("/kaggle/input/vjtitask/sample_submission.csv")


In [4]:
df2

,id,text
0,1041016773991,मोदी साहेब वरती चांगले तो पण त्यांच्या साईटला ...
1,109362481297,In #Jawan you will get: 1. 1st class action 2....
2,985019053532,किती दहशत आहे दोन्ही पवारांची बारामती कर मोकळे...
3,436629695381,आजवर का नाही विकास केलात मित्रा घरात किती प्रत...
4,585196067684,घराणेशाही थांबवायची असेल तर मोठ्या कॉलेज विद्य...
...,...,...
1995,148770317688,", अरे चूतीया, रोजगार उपलब्ध करून देणे सरकारच न..."
1996,288582831883,समाजातील गरीब लोकांसाठी मदतीला धावून येणारा एक...
1997,285086247879,Yuvraj is one of the most underrated players i...
1998,114758927004,"If muslim,sikh can wear their relegious attire..."


In [3]:
df

,id,text,complaint,demands,praise,questions
0,500796286320,Wow! From what I've observed from this documen...,0.0,0.0,1.0,0.0
1,838906157157,काय रे dungnat मेंदु असणाऱ्या आंधभक्ता तुझा आई...,1.0,0.0,0.0,0.0
2,1011026626743,अजित दादा आणि प्रफुल्ल पटेल यांनी केलेल्या काम...,0.0,0.0,1.0,0.0
3,1068853499446,"She's saying that ""doing her own research"" led...",1.0,0.0,0.0,0.0
4,502772748919,"That is not Karen, that is perfectly reasonabl...",1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
3995,815979127763,Why not just buy some oil platforms and start ...,0.0,0.0,0.0,1.0
3996,426508330840,अरे भाई हे तर सगळे हिंदीत बोलतात आणि राहता महा...,1.0,0.0,0.0,0.0
3997,1016442533101,सरकार ने जी योजना आणली आहे ती खरोखरच खूप चांगल...,1.0,0.0,0.0,0.0
3998,1096741594952,मुख्यमंत्री एकनाथ शिंदे यांच्या नेतृत्वाखाली म...,0.0,0.0,1.0,0.0


In [6]:
#df2['language'].unique()

In [7]:
#df['language'].unique()

In [8]:
#def is_hinglish(text):
    #hinglish_keywords = ['badiya','kya','kaise', 'bahut','raha']
    #for word in hinglish_keywords:
     #   if word in text.lower():
      #      return True
    #return False

#df1['language'] = df1.apply(lambda row: 'Hinglish' if is_hinglish(row['text']) else row['language'], axis=1)




In [9]:
#df1[df1['language']=="Hinglish"]

In [3]:
import pandas as pd
import random


text_examples = {
    "demands": {
        "marathi": [
            "मी दोन दिवसांपूर्वी जो फ्रीज ऑर्डर केला होता, तो अजून आला नाही. कृपया याची माहिती द्या आणि त्वरित डिलिव्हरी करा.",
            "आमच्या बिलाचे पेंडिंग पेमेंट आहे का ते मला कळवा आणि शक्य तितक्या लवकर बिलाचे निवारण करा.",
            "तुमच्या बँकेचे नवीन होम लोनचे दर काय आहेत? मला घर खरेदीसाठी वित्तपुरवठा हवा आहे.",
            "घराणेशाही थांबवायची असेल तर मोठ्या कॉलेज विद्यापीठ मध्ये विध्यार्थीसंघटना वाढवायला पाहिजेत आणि त्यांच्या निवडणुका चालू करायला पाहिजेत!",
            "माझ्या घरातील वीज पुरवठा दोन दिवसांपासून विस्कळीत आहे. कृपया याची तत्काळ दुरुस्ती करावी आणि मला पुढील २४ तासांमध्ये अपडेट पाठवावा.",
            "ऑर्डर केलेल्या उत्पादनाचे वितरण अजूनपर्यंत झाले नाही. मला ते उत्पादन खूप महत्त्वाचे आहे आणि कृपया आजच डिलीव्हरी पूर्ण करा.",
            "कृपया मला माझ्या ताज्या ऑर्डरच्या स्थितीविषयी माहिती द्या. मी ऑर्डर केलेल्या वस्तूची मला खूप गरज आहे, आणि तो उत्पादन मला लवकरात लवकर मिळावा, हे माझ्यासाठी खूप महत्त्वाचं आहे. जर कुठल्या कारणामुळे त्याची डिलीव्हरी लांबणीवर पडत असेल, तर कृपया मला कारण सांगावे आणि मी कसे पुढे जाऊ शकतो, हे सांगावे. मी अशा परिस्थितीत पूर्णपणे थांबू इच्छित नाही, आणि मला सर्वात जलद डिलीव्हरी पर्याय विचारात घेतल्यास हे कसे होईल, ते कृपया स्पष्ट करा.",
             "माझ्या मागील ऑर्डरचा पेमेंट अजूनपर्यंत प्रोसेस झाला नाही. माझा पेमेंट गेटवे योग्यपणे काम करत नाहीय, आणि मी त्या कारणामुळे आधीच किमान तीन वेळा पुनःप्रयास केला आहे. तुमच्या वेबसाइटवरील सर्व पेमेंट गेटवे समजून घेतल्यास, ते खूपच हळू आहेत आणि अनेक वेळा ते क्रॅश होतात. कृपया हे त्वरित दुरुस्त करा आणि मी तुमच्याकडून अपेक्षीत सेवा प्राप्त करू इच्छितो. त्याचबरोबर, मला इतर सुरक्षित आणि जलद पेमेंट गेटवे उपलब्ध करून द्या.",
             "तुमच्या दिलेल्या सेवेच्या दर्जा सुधारावा. मला खूप वेळा तुमच्याशी संपर्क साधावा लागला आणि जेव्हा ते सॉल्व होतं नाही, तेव्हा मी जास्त तणावात होतो. ग्राहक सेवेचे प्रतिनिधी अधिक ट्रेनिंग घेत असावेत, कारण मला खात्री नाही की ते समस्येचे योग्य समाधान देत आहेत. कृपया या संदर्भात कार्यवाही करा आणि मला खात्री देणे आवश्यक आहे की भविष्यात अशी समस्या उद्भवणार नाही.",
            "तुम्ही म्हणाला होता की शाळेतील क्रीडा मैदानासाठी नवीन साधनं आणणार. ती कधी येणार आहेत? मुलांना सराव करायचा आहे.","आमच्या भागात सार्वजनिक शौचालय नाही. कृपया काहीतरी उपाय करा, कारण स्वच्छतेची खूप गरज आहे."
        ],
        "hindi": [
            "सरकारी योजनाओं की जानकारी गांव के हर व्यक्ति तक पहुँचाने के लिए मोबाइल ऐप लॉन्च करें, जिससे लोगों को समय पर जानकारी मिल सके।",
            "स्थानीय बाजारों में छोटे दुकानदारों के लिए किराए में छूट दी जाए। उनकी आय महामारी के कारण बुरी तरह प्रभावित हुई है।",
            "मुझे मेरी सेवाओं के बारे में पूरी जानकारी चाहिए, जिसमें मेरे सभी खर्चे और उनके विवरण हो ताकि मैं अपने बजट को बेहतर तरीके से प्रबंधित कर सकूं।",
            "मेरी पिछले हफ्ते की ऑर्डर अभी तक डिलीवर नहीं हुई है। कृपया इसे तुरंत डिलीवर करें क्योंकि यह मेरे लिए अत्यधिक महत्वपूर्ण है।",
            "कृपया मेरी दरखास्त को प्राथमिकता के आधार पर प्रोसेस करें क्योंकि यह अत्यंत महत्वपूर्ण है और मुझे जल्द परिणाम चाहिए।",
            "आपकी डिलीवरी सर्विस बहुत धीमी हो गई है, और मुझे बार-बार डिलीवरी की स्थिति को लेकर पूछताछ करनी पड़ती है। मैंने जिस उत्पाद को ऑर्डर किया था, वह मुझे अब तक नहीं मिला है, और यह अब काफी ज़रूरी हो गया है। कृपया मेरे आदेश की स्थिति तुरंत स्पष्ट करें और मुझे यह सुनिश्चित करें कि डिलीवरी जल्दी होगी। मैं इस मुद्दे पर और अधिक विलंब नहीं सह सकता, क्योंकि इस उत्पाद का मेरे व्यवसाय के लिए अत्यधिक महत्व है। कृपया इसकी प्राथमिकता बढ़ाएं और मुझे जल्द से जल्द डिलीवरी का आश्वासन दें।",
            "मैंने आपके पास सेवा के लिए आवेदन किया था, लेकिन मैंने अब तक किसी भी तरह का उत्तर या प्रतिक्रिया नहीं प्राप्त की है। मैंने कई बार आपके कस्टमर सपोर्ट से संपर्क किया है, लेकिन हर बार मुझे बस यह कहा जाता है कि 'हम जल्द ही संपर्क करेंगे,' और फिर कोई कार्रवाई नहीं होती। क्या आप कृपया मेरी इस आवेदन पर कार्यवाही करेंगे और मुझे त्वरित प्रतिक्रिया देंगे? मुझे लगता है कि आपकी सेवा में सुधार की जरूरत है, ताकि ग्राहक आसानी से संपर्क कर सकें और समस्याओं का समाधान हो सके।",
            "मेरे पास एक मोबाइल फोन का मुद्दा है जो बिल्कुल ठीक से काम नहीं कर रहा है। मैं ने पहले ही तकनीकी सहायता से संपर्क किया है, लेकिन अभी तक कोई हल नहीं मिला। फोन की बैटरी बहुत जल्दी खत्म हो जाती है, और नेटवर्क भी बार-बार कट जाता है। कृपया जल्दी से जल्दी इसे ठीक करवाने की व्यवस्था करें, क्योंकि यह अब मेरे लिए एक महत्वपूर्ण कार्य में रुकावट पैदा कर रहा है। मुझे उम्मीद है कि आप जल्द ही इस मुद्दे पर ध्यान देंगे और इसे समाधान करेंगे।",
            "Can you please forward me the notes from yesterday’s lecture? I couldn’t attend because of a family emergency, and I need to catch up before the next class.",
            "यार, मुझे कल की मीटिंग के minutes urgently चाहिए. Manager ने पूछे हैं, और मेरे पास कोई details नहीं हैं. प्लीज जल्दी से भेज दो.",
            "Hey, can you help me book tickets for the weekend trip? I’m stuck at work and don’t have time to check the schedules.",
            "तुम्हारे पास maths की वो reference book है ना? मुझे एक equation समझने में problem हो रही है. प्लीज आज ही दे दो.",
            "Can you remind the landlord to fix the water leakage? It’s been almost a week, and it’s only getting worse. Please follow up with him today.","Yaar, humare building mein jo lift hai, wo har doosre din band ho jaati hai. Aise mein kya humein chadhne ke liye stairs ka use karna padega? Jaldi se repair karwa do."
        ],
        "english": [
            "I have been waiting for the furniture delivery for over two weeks now. Please provide an update and ensure it reaches this week.",
            "My internet connection has been down since yesterday evening. Kindly send a technician to fix it urgently.",
            "The air conditioning in the conference room is not working properly. Can you have it serviced before the next meeting?",
             "Maine do din pehle courier service ke liye request ki thi, abhi tak koi update nahi aayi. Jaldi se mujhe status batao.",
             "Yeh AC pura properly kaam nahi kar raha hai. Please technician bhejo aur repair karwa do.",
             "Kya aap mujhe mere electricity bill ke detailed breakdown de sakte ho? Kuch amounts galat lag rahe hain.",
             "Mujhe pata chala ki aapke paas ek naya cashback offer chal raha hai. Mujhe iska benefit kaise milega, please explain karo.",
             "Tujhe office supplies urgent chahiye kya , kal tak deliver kar sakti kya?",
            "Mujhe mera latest bill ka pura breakdown chahiye, jisme saare charges aur reasons detail mein diye hoon. Please jaldi bhejiye.",
             "Meri order ab tak deliver nahi hui hai. Mujhe aaj urgent kaam ke liye zarurat hai, toh please fast delivery karwa dijiye.",
            "I have been waiting for my furniture delivery for over two weeks now. I was initially promised a delivery date, but the product has not yet arrived. I am not sure if it’s an issue on your end or with the delivery company, but I would like to know the specific reason for this delay. This has been very frustrating as I’ve been trying to plan around the delivery, but each time I contact customer support, I get vague responses. Could you please provide a detailed explanation and an updated delivery date? I need this matter resolved as soon as possible.",
            "I ordered a new mobile phone two weeks ago, and I have not received it yet. I am not sure where the delivery is being delayed, but it has been long enough now for me to get concerned. The tracking information is not updated frequently, and I feel like I’m being left in the dark. Can you please provide me with an exact timeline on when I can expect my phone? Additionally, I would like to request an expedited shipping option, as this delay has already caused me inconvenience.",
            "My internet connection has been down for over 24 hours, and I have not been able to use my services at home for work purposes. This is extremely frustrating, as I rely heavily on my internet connection for both work and personal matters. I’ve tried troubleshooting on my end, but it seems like the problem is with your service. Could you please send a technician as soon as possible to inspect and repair the connection? I would also appreciate it if you could offer some compensation for the prolonged outage.",
            "Arey yaar, kal jo match ke tickets lene bole the, woh kab arrange karoge? Abhi tak kuch update nahi diya."
            "Bro, mujhe aaj ka assignment bhej de na, kal submit karna hai aur maine banaya nahi.",
            "Didi, woh tuition ke liye jo naye books aaye hain, kya mujhe unka ek set arrange kar ke de sakti ho?",
            "Arey yaar, yeh Ganpati pandal ke decor ke ideas de de, Tujhe ideas kaafi ache aate hain aur pls ganpati ka booking bhi tu hi karde .",
            "Tu kal office se jaldi aa sakta hai kya? Plan karte hain Marine Drive walk ka bhot dino se hum mile nhi lets spend some qualtiy time.",
            "Oye, tu woh group discussion ke liye points bana ke ready kar na. Kal woh tough sir ke saamne presentation hai. Agar usne kuch ulta question poocha toh kaise handle karenge? Main bhi apna part kar raha hoon, but teamwork zaroori hai yaar.",
            "Bro, can you please help me with the notes from last week’s lecture? I missed it because of the train delay, and I’m totally clueless about tomorrow’s test. Just click pictures and send them on WhatsApp; it’ll be a huge favor, yaar. Don’t forget, okay?"
        ],
        "Malyalam":[
            "എനിക്ക് കഴിഞ്ഞ ആറുമാസത്തിനുള്ളിൽ നടന്ന ഇടപാടുകളുടെ വിശദമായ വിവരങ്ങൾ ആവശ്യമുണ്ട്. ഓരോ ചെലവും എന്തിനാണ് വന്നതെന്ന് വ്യക്തമാക്കുക.",
            "വീട് നവീകരണത്തിനായി അനുവദനീയമായ വായ്പയുടെ എല്ലാ വിശദാംശങ്ങളും അയക്കുക. അതിൽ നിന്നും ഏറ്റവും നല്ല ഓഫർ തിരഞ്ഞെടുക്കാൻ ഞാൻ സഹായം ആവശ്യമുണ്ട്.",
            "माझ्या बरोबर स्टेशनवर येशील का? मला तिथून काही सामान आणायचं आहे आणि ते खूप जड आहे. एकट्याने न्यायचं तर अवघड आहे. कृपया थोडं मदत करशील का?",
            "यार, मुझे urgent notes चाहिए अगले हफ्ते के लिए. तुमने जो summary बनाई थी, वो भेज दो. मैं वादा करता हूं कि तुम्हारा नाम presentation में जरूर mention करूंगा.",
            "Can you help me complete the project draft by tonight? I’m stuck on the analysis part, and you’re way better at it. Let’s split the sections and finish this together.",
            "तुझ्याकडे जर चार्जर असेल तर मला देशील का? माझा फोन पूर्णपणे डिस्चार्ज आहे, आणि मला ऑफिसच्या कामासाठी चार्ज करायचं आहे.",
             "क्या तुम प्लीज मुझे train schedule check करके बता सकते हो? मुझे urgent काम से बाहर जाना है, और time सही से पता नहीं चल रहा है.",
             "Can you please arrange for a bigger classroom for our weekly club meetings? The current one is too small for everyone to fit in comfortably. We’ve been requesting this for a while now, and it’s important for better participation and interaction during discussions.",
              "क्या आप हमारे स्कूल के playground को repair करवाने की व्यवस्था कर सकते हैं? बच्चे खेलते समय चोटिल हो रहे हैं क्योंकि मैदान uneven है और कई जगह गड्ढे भी हैं. यह urgent matter है, कृपया इस पर ध्यान दें.",
             "तुम्हाला जमल्यास लायब्ररीच्या वेळा वाढवू शकता का? परीक्षा जवळ येत आहेत, आणि अभ्यासासाठी थोडा जास्त वेळ लागतो. जर हे शक्य असेल, तर खूप विद्यार्थ्यांना फायदा होईल.",
             "The cafeteria menu needs an upgrade. Can we have more healthy and budget-friendly options? Many of us have been finding the current menu repetitive and a bit unhealthy for everyday consumption.",
             "आप लोग community park की सफाई regular basis पर क्यों नहीं करते? यह पार्क सबके use के लिए है, और हर जगह गंदगी जमा हो रही है. कृपया सफाई को schedule में डालें.",
             "आम्ही गणपती मंडळासाठी यंदा नवीन मंडप उभारायचा विचार करतो आहोत. त्यासाठी कचरा उचलणे, लाईट्स लावणे, आणि जागा स्वच्छ ठेवण्याच्या तुमच्या सेवांची आवश्यकता आहे. कृपया लवकर उत्तर द्या."
        ],
        "gujrati":[
            "વિજળી કાપના કારણે મારી ઓફિસનું કામ પ્રભાવિત થયું છે. કૃપા કરીને પાણી અને વિજળીની સમસ્યાનું તરત નિરાકરણ કરો."
        ]
    },
    "questions": {
        "marathi": [
            "सरकारने नुकत्याच मंजूर केलेल्या कृषी विधेयकांचा शेतकऱ्यांच्या उत्पन्नावर कसा परिणाम होईल?",
            "सार्वजनिक वाहतूक व्यवस्था सुधारण्यासाठी स्थानिक प्रशासन कोणती धोरणे आखत आहे आणि त्याची अंमलबजावणी कधी होईल?",
            "ग्रामीण भागातील विद्यार्थ्यांना डिजिटल शिक्षण तंत्रज्ञान उपलब्ध करून देण्यासाठी कोणती पावले उचलली जातील?",
           "क्याआप बता सकते हैं कि मैंने पिछली बार जो कंप्लेंट दर्ज की थी, उस पर क्या एक्शन लिया गया?",
            "तुमच्या नवीन योजनांचा कसा फायदा होईल? मला कोणते पर्याय उपलब्ध आहेत याची अधिक माहिती हवी आहे.",
            "जर मी माझ्या आरोग्य विम्याचा दावा करायचा असेल तर प्रक्रिया काय आहे आणि मला कोणते कागदपत्र लागतील?",
            "तुम्ही दिलेल्या अलीकडील योजनांबद्दल मला थोडं अधिक माहिती हवी आहे. कशाप्रकारे आणि कोणत्या पद्धतीने ती माझ्या व्यवसायासाठी फायदेशीर ठरेल? मला त्या योजनांच्या अटी, नियम आणि अनुप्रयोग प्रक्रियांची संपूर्ण माहिती पाहिजे. हे समजून घेणे माझ्यासाठी महत्त्वाचे आहे, कारण मी त्याचा सर्वोत्तम वापर करून, माझ्या कंपन्याला वाढवू इच्छितो. कृपया यावर लवकर कार्यवाही करा.",
            "तुमच्या सेवा उत्कृष्ट आहेत, आणि मी तुमच्या टीमला समर्पणाच्या कामाबद्दल सल्ला देतो. तथापि, माझ्या काही गोष्टींसाठी अतिरिक्त माहिती मिळवण्याची आवश्यकता आहे, जसे की काही वेळांमध्ये विशेष बदलांची मदत कशी होईल, तसेच व्यवसायाच्या डिझाइनसाठी पुढील कदम.","सर, पुढच्या आठवड्यात स्पर्धेचा अंतिम फेरीचा कार्यक्रम आहे का? वेळ आणि जागेबद्दल कळवा."
        ],
        "hindi": [
            "नई शिक्षा नीति से ग्रामीण बच्चों की शिक्षा में किस प्रकार के बदलाव देखने को मिलेंगे?",
            "सरकारी अस्पतालों में संसाधनों की कमी को पूरा करने के लिए कौन सी नई योजनाएँ लागू की जा रही हैं?",
            "क्या शहरी क्षेत्रों में पानी के संकट को हल करने के लिए दीर्घकालिक योजना बनाई जा रही है?",
            "क्या आपकी टीम मुझे बता सकती है कि अगर मेरा ऑर्डर डिलीवर नहीं हुआ है तो मुझे क्या करना चाहिए और उसकी प्रक्रिया कितनी लंबी होती है?",
            "आपके द्वारा पेश की गई योजनाओं में कौन-कौन से फायदे हैं और मुझे किसके लिए आवेदन करना चाहिए?",
            "क्या कोई तरीका है जिससे मैं अपने बैंक अकाउंट की डिटेल्स जल्दी अपडेट कर सकूं और कोई समस्या न हो?",
            "क्या मुझे इस साल के अंत तक आप द्वारा प्रस्तावित योजनाओं के परिणामों के बारे में अधिक जानकारी मिल सकती है? मेरे पास कुछ ऐसे प्रश्न हैं जो मेरी योजनाओं में प्रभावी रूप से योगदान कर सकते हैं, और मैं चाहता हूँ कि यह योजना जल्द से जल्द कार्यान्वित हो। क्या आप मुझे योजनाओं के बारे में विशिष्ट जानकारी दे सकते हैं, और क्या कोई विशेष परिस्थिति है जिसे मुझे ध्यान में रखना चाहिए?",
            "क्या आप मेरी पिछली सेवा समीक्षा के बारे में कुछ और जानकारी दे सकते हैं? मैंने अपनी पिछली सेवा से जुड़े कुछ सवाल पूछे थे, लेकिन मुझे अभी तक कोई स्पष्ट उत्तर नहीं मिला। कृपया मुझे इसका अधिक विस्तृत विवरण दें ताकि मैं इस बारे में अपने निर्णय पर विचार कर सकूं।",
            "क्या तुम्हें पता है, इस बार गणपति के लिए Lalbaug cha Raja कब open होगा? मैं हर साल जाता हूं, लेकिन अभी तक dates confirm नहीं हैं. क्या process है pass लेने का, और कितनी भीड़ होने वाली है?"

        ],
        "english": [
            "Why is the warranty on the product void when I clearly followed the usage instructions",
            "Can you explain how the rewards points system works and how I can redeem them",
            "was the vibrator supposed to do that or i went overboard with it....did i get a little too freaky this time..ughhh",
            "What are the steps involved in applying for a loan extension? Are there any additional charges?",
            "Can you provide some insight into why the delivery was delayed and what measures are being taken to prevent it in the future?",
            "Woh jo offer kal advertise kiya tha, woh abhi bhi available hai kya ya expire ho gaya?",
            "Mere account mein late fee ka charge kyun add kiya gaya hai? Maine payment time pe kiya tha na.",
            "Kya mujhe pata sakte ho ki customer care support ka fastest response time kitna hai?",
            "The reason geese are still in central saskatchewan is because there's no snow and they still have food with the grain on the ground. The Canada Goose is probably best known for its fall migration, so in September and October, these guys will be flying south from their native land of Canada to overwinter in the United States. The reason they do this migration is primarily for food availability.",
            "Mujhe samajh nahi aa raha ki yeh EMI calculation kaise hua hai. Kya aap mujhe explain karenge?",
            "do you see any media houses on reddit, or celebrities or reddit being mentioned in any commercial mode of communication? do your parents/grandparents know about reddit? but (mostly) they ll know about facebook, instagram. thatâ€™s what is mainstream. reddit has about 50 million daily users while instagram has over 500 million daily users, so it should be self explanatory but nvm ðŸ’ðŸ»â€â™€ï",
            "You sure that helps? Because I can't even walk that's how bad it made me feel . Low back pain is unbelievable. Muscles, headaches and I can't sleep without sleeping med. Clanapam wich is probably not good for me to take. Originally they proscribed this med. For heart palpation. Now I know whycI had heart palpation- elevated blood pressure. Don't we learn a lot just from interacting here .!!!",
            "Could you provide a breakdown of how your new product feature will benefit me in the long run? I am really interested in understanding its potential advantages and whether it’s suitable for my needs. I have seen the initial product description, but I would like more detailed information on how it works, the technical specifications, and its compatibility with existing products. Can you explain how these new features will directly address my needs and make my current processes more efficient?",
            "Could you let me know what steps are involved in applying for an extension on my subscription plan? I want to make sure that I understand the entire process, including any additional charges or requirements. Additionally, I would appreciate it if you could let me know if there are any promotions or discounts that I could use when extending my plan, as I am trying to make this a cost-effective decision. Could you provide a timeline for when this extension can be processed?",
            "Yaar, kya tujhe pata hai ki Kalaghoda Arts Festival kab start hota hai? Main abhi tak schedule nahi dekh paaya. Kya iss baar bhi woh street performances aur workshops honge? Last time kaafi interesting laga tha, toh soch raha hoon iss baar pura explore karoon.",
            "Do you know the process for applying for the advanced workshop on AI and Robotics? Where should I register?",
            "Acha sun, kal ka group discussion main kaun-kaun lead karega? Mujhe time pe jaana hai toh plan kar raha hoon.",
            "Can you clarify the grading criteria for the final project? Are there specific weightages for creativity, content, and presentation? Understanding this will help us prepare better and ensure we meet expectations.",
            "क्या आप मुझे बता सकते हैं कि इस सप्ताह के अंत तक स्कूल की छुट्टियां कब से शुरू होंगी? हमें अपनी योजना बनाने में मदद मिलेगी।",
            "Could you explain how the new feedback system works? Are we supposed to submit it online or in person? I just want to make sure I follow the correct process",
            "माझ्या मुलासाठी स्कूलच्या प्रवेशासाठी अर्ज कसा करावा हे मी समजून घेऊ इच्छितो. सरकारने शाळांमध्ये विविध योजनांचा कार्यान्वयन सुरू केले आहे का? कोणती योजना माझ्या मुलासाठी उपयुक्त असेल? शिक्षणाचा स्तर वाढवण्यासाठी प्रशासनाच्या पुढील पावलांविषयी मला माहिती पाहिजे. शाळेच्या दाखल्यांमध्ये कोणत्या विशिष्ट गोष्टींची आवश्यकता आहे? आम्हाला सहकार्य करण्यासाठी प्रशासनाकडून योग्य माहिती मिळवली पाहिजे. कृपया मार्गदर्शन करा, आम्ही योग्य निर्णय घेऊ शकू."

        ],
        "gujrati":[
            "મારા હોમ લોન માટેની નવી યોજનાનો લાભ કેવી રીતે લેવો અને મને કેટલું વ્યાજ ભરવું પડશે?"        ]
    },
    "complaint": {
        "english": [
            
            "My internet connection has been down since yesterday evening. Kindly send a technician to fix it urgently.",
            "Well, that driver needs to penalized heavily for this. There are plenty of warnings. He can see the yellow flags flashing, the car on the side of the track, the car in the middle of the track and this is not like a blind corner where you are surprised by something appearing in front of you, it's a straight, plently of time to slowdown. He has no excuses for going that fast in such a situation.",
             
            "Bhai, mera bill kuch jyada ho gaya hai, kuch extra charges the. Kya karna padega?",
            "Kuch bhi!! The guy is talking about battling misinformation and here you are making another bs claim about how youâ€™ve travelled 300 kms and cast your vote..unless there has been a secret ballot held for your constituency or just for you perhaps pray tell me where exactly did you go vote..or perhaps you misunderstood and you thought he meant voting on reality showsðŸ¤·ðŸ»â€â™‚ï¸I mean come on man seriously" ,
            "Yaar, tumhari delivery service bahut slow hai. Ek toh koi updates nahi mil rahe. band hi kardo tum  apna yeh bussiness",
            "Bhai, app baar baar crash ho rahi hai, kuch kaam karo yaar mera bhot important text ane wala hai ispe.",
            "Mam is so not fair with us on the selection for the club mentors, she's only judging people based on their CGPA and not their skills so fucking unreasonable",
            "Tu bata, Mumbai local ka crowd din ba din itna zyada kyun ho raha hai? Saans lena mushkil hota hai.",
            "If muslim,sikh can wear their relegious attire freely then why it becomes an issue if hindu do same , Is this equality On this comment there are mainly muslim ,sikhs saying about the future of these kids care about your future they will surely become nice educated people Not like you guys partilly favour to your religion . Kattar hindu hu ye sab nahi shunga mere dharam pe alank lagane wallo ka",
            "It seems the video is made to satisfy Modi Bhakts who are outside Maharashtra. Situation is very different in Maharashtra. People don`t attend Shinde and BJP`s Rally. BJP pays people to come to event and people run away while Bhashaan is going on .... BJP didn`t contest Andheri seat. They are scared to do so .... Also Modi is scared to contest BMC election, which is vacant from last 1 year ....",
            "The lecture today was so confusing! The professor didn’t even explain the concepts properly and kept skipping slides. How are we supposed to understand the syllabus at this pace? We should talk to him and request more detailed classes.",
            "Shazam bro I am your big fan if you see this msg pls reply me I have been using 14 pro for 5 months I had a school trip 3 weeks ago on that trip we had a dj program after that my camera had fully gone camera is not working I donâ€™t no what to do ðŸ¥ºðŸ’” I am from a middle class family 5 months pattiye pole Pani eduth vaangiya phone aayirunnu ippo moonji irikaan is there any way for solving this problem",
            "Classroom की condition देखी है तुमने? Fans तो चलते ही नहीं हैं, और गर्मी में बैठना मुश्किल हो जाता है. ये AC कब repair होगा, कुछ पता है क्या? हमने कई बार complaint दी है, लेकिन कोई action नहीं लिया गया."
        ],
        "hindi": [
            "आपकी डिलीवरी सर्विस बहुत धीमी है। मुझे बहुत इंतजार करना पड़ा।",
            "ऑनलाइन सेवेच्या वेळापत्रकात नेहमी गडबड होत असते. हे कधी सुधारणार?",
            "Maine 3 din pehle order kiya tha, abhi tak mujhe delivery ka koi update nahi mila.",
            "The payment portal on your website is too slow and keeps crashing.",
            "तुमच्या कर्मचारी कर्मचार्‍यांचा वागणूक अत्यंत रुखा आहे. कृपया त्या बाबीवर लक्ष द्या.",
            "तुमच्या वितरण सेवा खूपच धीमा आहे. माझ्या ऑर्डरची स्थिती तपासा.",
            "तुमच्या वेबसाइटवर बार-बार क्रॅश होतो आहे. यावर उपाय शोधा.",
            "गेल्या महिन्यात मी वीज बिल भरलं होतं, पण तरीही मला परत बिल आलं आहे. कृपया दुरुस्त करा.",
            "तुमचा कॉल सतत डिसकनेक्ट होतो. नेटवर्कची समस्या काही केल्या संपत नाहीये.",
            "काल मी गाडीतून प्रवास करताना खड्ड्यांमुळे त्रास झालो. रस्त्यांची अवस्था सुधारावी.",
            "पाणीपुरवठा खंडित झालाय, सकाळपासून पाणी नाही. लवकरात लवकर पुरवठा चालू करा.",
            "तुमच्या स्टॉपवर रिक्षा नेहमी उपलब्ध नसते. काही पर्याय आहे का?",
            "मोदी साहेब वरती चांगले तो पण त्यांच्या साईटला आजूबाजूला सगळे काम करणार आहेत ना ती सगळी स्वतःच्या किसा भरणारी आहेत आणि त्यामुळे आम्हाला फक्त मोदी साहेबांमुळे फक्त बघून ह्या लोकांना मला मत कमळायला टाकू वाटते मोदी साहेबांना यांना पण त्यांची सगळी दोन नंबरची प",
             "The road near our colony is in such bad condition that it takes an extra 30 minutes to reach the station. When will the authorities fix this mess?",
             "Classroom में हर दिन पानी की दिक्कत हो रही है. Water cooler खराब पड़ा है, और कोई action नहीं लिया जा रहा. इतनी basic चीज़ भी manage नहीं कर सकते क्या?",
            "किती दहशत आहे दोन्ही पवारांची बारामती कर मोकळे बोलू पण शकत नाहीत.त्यांना कोणा एकट्याचे उघड समर्थन देखील करू शकत नाहीत. खरंच बारामती मध्ये खूप दहशत दोन्ही पवारांची हे लक्षात येते",
             "माझ्या कॉलनीमधल्या कचऱ्याच्या टाक्या खूप भरल्या आहेत, आणि अजून कोणी उचलायला आलेलं नाही. यामुळे परिसरात दुर्गंधी पसरली आहे. तात्काळ काहीतरी करायला हवं.",
             "Why is the gym AC not working for the past week? It’s so uncomfortable to work out in this heat. I’ve complained twice, but no action has been taken.",
            "Library में books का arrangement इतना खराब है कि कोई भी book time पर नहीं मिलती. Staff को system improve करना चाहिए ताकि students को convenience हो.",
           "तुम्हें पता है, hostel का खाना अब बिल्कुल खराब हो गया है. हर दिन वही oily और ठंडा खाना मिलता है. कुछ बोलना पड़ेगा warden को.",
           "The streetlights in our area haven’t been working for a week now. It’s so unsafe at night. Someone should report this to the municipal office.",
           "Classroom में projector फिर से खराब हो गया है. Presentation देना impossible हो जाता है. IT department कभी timely repair क्यों नहीं करता?",
           "Why does the bus always run late in the mornings? It’s making everyone late for work. This needs to be addressed immediately.",
         "Library में WiFi इतना slow है कि कोई भी resource download नहीं हो पाता. ये तो students के लिए basic necessity है, और इसे जल्दी ठीक होना चाहिए.",
            "The gym equipment in our society is in poor condition, with several machines not functioning properly. Despite repeated complaints, nothing has been fixed. It’s high time this issue is addressed.",
            "हमारी बिल्डिंग में पानी की सप्लाई पिछले कुछ दिनों से अनियमित हो गई है. कभी पानी आता है, कभी नहीं. प्लीज इसकी जांच करके इसे तुरंत ठीक करें.",
           "Yaar, jo tiffin aaj college ke canteen se liya tha, usme khana ekdum stale tha. Complain karni padegi.","Classroom ke fans repair karne ke liye ab tak koi technician nahi aaya. Bahut zyada garmi ho rahi hai.",
            "शाळेतील पिण्याच्या पाण्याच्या टाक्या अजूनही साफ झाल्या नाहीत. त्यामुळे विद्यार्थ्यांच्या आरोग्याला धोका आहे. कृपया पाण्याच्या स्वच्छतेसाठी काहीतरी ठोस उपाय करा.",
            "रात के समय बिजली बार-बार कट रही है, जिससे हम काम नहीं कर पा रहे हैं। कृपया इस समस्या का समाधान जल्द से जल्द करवाइए। यह बहुत परेशानी का कारण बन रहा है।",
 "The public transport service has been consistently delayed over the last few weeks. I missed important appointments because of this. Can we expect any improvements or alternatives to prevent this from continuing?",
 "Bhai, mere apartment ka parking area jo hai, wo bohot messy ho gaya hai. Log apni car idhar-udhar park karte hain. Humein kuch manage karke parking ka system fix karna padega.",
            "मैंने आपकी कंपनी से एक लैपटॉप खरीदा था, लेकिन जब उसे खोला तो उसमें कुछ खामियां थीं। स्क्रीन पर धब्बे थे और बैटरी की बैकअप भी बहुत कम थी। मैंने ग्राहक सेवा से संपर्क किया, लेकिन मुझे कोई उचित समाधान नहीं मिला। मुझे उम्मीद थी कि आपकी कंपनी उच्च गुणवत्ता वाले उत्पाद प्रदान करेगी, लेकिन अब मुझे बहुत निराशा हो रही है। कृपया इस समस्या को जल्द हल करें और मुझे रिप्लेसमेंट या रिफंड की प्रक्रिया के बारे में जानकारी दें।",
"मुझे आपके द्वारा दी गई इंटरनेट सेवा से बहुत परेशानी हो रही है। पिछले कुछ दिनों से इंटरनेट की स्पीड बहुत धीमी हो गई है, और कई बार तो सर्विस बिल्कुल भी काम नहीं कर रही। मैंने कई बार शिकायत की है, लेकिन कोई ठोस समाधान नहीं मिला। यह मेरे लिए बहुत ही असुविधाजनक है क्योंकि मैं वर्क फ्रॉम होम करता हूं और इस सेवा की अत्यधिक आवश्यकता है। कृपया इसे जल्द ठीक करें और मेरी समस्या का समाधान करें।",

"मैंने कुछ दिन पहले आपके स्टोर से एक जोड़ी जूते खरीदी थी, लेकिन जब मैंने उसे पहना तो उसे बहुत असुविधा महसूस हुई। जूते की क्वालिटी बिल्कुल खराब थी और उसमें से बहुत तेज़ गंध आ रही थी। मैंने उसे वापस करने की कोशिश की, लेकिन स्टोर में किसी ने मेरी मदद नहीं की। अब मुझे यह महसूस हो रहा है कि आपने मुझे दोषी करार दिया है। कृपया इसे शीघ्र सुलझाएं और मुझे अपनी समस्या का समाधान दें।",
            "Maine tumhari gym membership li thi aur jo services mujhe mile, wo bilkul bhi satisfactory nahi thi. Gym ki cleanliness pe bahut dikkat thi, aur equipment bhi out of order the. Jab maine complaint ki, toh staff ne sirf fake assurances diye, lekin kuch bhi improve nahi kiya gaya. Aise kaise chalega? Agar tumhare gym ka standard itna low hai, toh mujhe apni membership refund karni padegi. Mujhe better services ki ummid thi. Please jaldi solution do, warna main koi aur gym join karunga.",

"Tumhari food delivery service ne mujhe kaafi disappoint kiya hai. Mainne ek pizza order kiya tha, lekin jab wo aaya toh wo bilkul cold tha. Uska taste bhi kuch aur hi tha, bilkul fresh nahi laga. Maine customer service se contact kiya, lekin unhone koi proper explanation nahi diya. Aise kaise chalega? Mujhe toh lagta hai ki tumhare delivery process ko improve karne ki zarurat hai. Agar mujhe baar baar aise issues face karne padenge, toh main tumhe recommend nahi karunga. Jaldi solution chahiye.","I’m facing a serious issue with my mobile phone, as it keeps freezing frequently, even after multiple resets. I need to use important apps for work, but this issue is making it almost impossible. I reached out to customer service, but they haven’t been helpful at all. It seems like they don’t understand the urgency. Please provide a proper solution or replacement, as this has been very disruptive for my daily tasks. I would appreciate it if this issue is resolved at the earliest.",
            "सरकारच्या सार्वजनिक वाहतूक सेवांमध्ये प्रचंड अडचणी आहेत. बसेस वेळेवर धावत नाहीत आणि प्रवाशांना अतिरिक्त शुल्क आकारले जाते. सरकारने सार्वजनिक वाहतूक प्रणालीमध्ये सुधारणा केली पाहिजे, ज्यामुळे प्रवास सुलभ आणि किफायतशीर होईल.",
            "The train services at the railway station are frequently delayed. Passengers do not receive proper information, and delays have become a common occurrence. The government should improve the information systems at stations and make the train services more reliable.",
            "सरकारच्या शाळांमध्ये शिक्षकांची कमतरता आहे. विद्यार्थ्यांची संख्या खूप जास्त आहे आणि शिक्षकांचा अभाव आहे. काही विषयांमध्ये विशेष तज्ञ शिक्षकांनाही भरण्यात आलेले नाही. यामुळे शिक्षणावर नकारात्मक परिणाम होतो आहे. सरकारने अधिक शिक्षकांची नियुक्ती करण्याची आवश्यकता आहे.",
            


        ],
        
    },
    "praise":{
        "english":[
            "Aapke kaam ka impact profound hai. Aap extraordinary ability rakhte ho ideas ko life mein lane aur boundaries push karne mein. Aapka enthusiasm aur passion ke saath kaam karna ek pleasure hai.",
            "Aapka leadership aur vision ne humare projects ko naye heights tak le gaya hai. Team ko motivate aur guide karne ki aapki ability remarkable hai. Aapka dedication aur unwavering commitment to excellence ke liye hum deeply appreciate karte hai.",
            "तुमची वक्तृत्वकला आणि प्रेझेंटेशन कौशल्य खूपच प्रभावी आहे. तुमचा आत्मविश्वास आणि स्पष्टता खूप प्रेरणादायक आहे.",
            "Mummy, aapki cooking amazing hai! Sab friends hamesha tareef karte hain aur bolte hai ki aapke jaisa khana aaj tak nahi khaya unhone hehehe.",
            "Your explanation on the new topic in today’s lecture was very clear. It’s rare to see this level of clarity.We need such proffesors like you who can make learning physics this easy"
            "Yaar, tu har sports event mein lead karta hai. Teri energy aur commitment ekdum top-notch hai!..You should  be a proffesional athlete mere bhai tu india ka naam roshan karega dekh lena meri intution hai",
             "तुझ्या स्केचिंगचं कौशल्य खरंच अप्रतिम आहे. तुला प्रत्येक नजरेचं बारकाईने निरीक्षण करण्याची कला आहे.","Tere dancing ka jo ekdum smooth flow hai na, woh crowd ko ekdum engage kar leta hai. Amazing!",
            "तुम्ही केलेली कामं पाहून खूप आनंद झाला. तुमच्या प्रयत्नांमुळे शाळेत एक नवीन ऊर्जा निर्माण झाली आहे. तुमच्या योगदानामुळे सर्व विद्यार्थ्यांना नवीन शिकण्याची प्रेरणा मिळाली आहे. तुम्ही जी मेहनत घेत आहात, ती खूपच प्रेरणादायी आहे. तुमच्या या कौशल्यामुळे अनेक लोकांना मदत होईल. तुमचे काम एक आदर्श ठरले आहे, आणि तुमचं समर्पण आणि मेहनत खूप कौतुकास्पद आहे.",
            "तुमचं काम खूपच उत्कृष्ट आहे! तुम्ही सगळ्या समस्यांचे समाधान शोधून त्यावर काम केलं आहे. तुम्ही केवळ उत्कृष्ट कार्य केलेच नाही, तर इतरांना देखील मदतीचा हात दिला आहे. तुमचं समर्पण आणि मेहनत नेहमीच उदाहरण ठरते.",
            "तुम्ही गेल्या काही महिन्यांत केलेल्या कामामुळे आम्हाला खूप सकारात्मक बदल दिसले. तुमचं नेतृत्व खूप प्रेरणादायक आहे, आणि तुमच्यामुळे सर्वांना योग्य दिशा मिळाली आहे. तुम्ही दिलेल्या मार्गदर्शनामुळे सगळ्यांचा आत्मविश्वास वाढला आहे.",
            "तुमचं काम फारच प्रभावशाली आहे. तुमच्या मेहनतीमुळे अनेक लोकांचं जीवन सुधारलं आहे. तुम्ही जे काम करत आहात, त्याची समाजाला खूप आवश्यकता आहे. तुमचं कार्य निश्चितच इतरांना प्रेरित करेल.",
            "तुमच्या कौशल्याने आणि ज्ञानाने खूप लोकांना मदत केली आहे. तुमच्या दिलेल्या मार्गदर्शनामुळे अनेक जण उत्तम कार्य करीत आहेत. तुमचं काम हे खरंच एक आदर्श आहे. तुमच्यामुळे खूप लोकांच्या जीवनात सकारात्मक बदल झाला आहे.",
            "आपके द्वारा किए गए कार्य ने सचमुच हम सबको प्रेरित किया है। आपकी मेहनत और समर्पण ने टीम को एक नई दिशा दी है। आपने न केवल उत्कृष्ट परिणाम हासिल किए हैं, बल्कि दूसरों को भी सीखने और आगे बढ़ने के लिए प्रेरित किया है। आपका कार्य सभी के लिए एक आदर्श बन गया है।",
            "आपका काम शानदार है! आपने जो भी किया, उसे न केवल बखूबी अंजाम दिया, बल्कि दूसरों को भी मदद की है। आपका समर्पण और परिश्रम वाकई काबिले तारीफ है। आपके योगदान ने सबको एक नई दिशा दी है।",
            "आपकी मेहनत और समर्पण ने संस्था में कई सकारात्मक बदलाव लाए हैं। आपके मार्गदर्शन से कई लोग अपनी क्षमताओं को पहचान पाए हैं और अपने कार्य में सुधार कर पा रहे हैं। आपके कार्य से हम सभी को प्रेरणा मिलती है।",
            "आपका काम अत्यधिक सराहनीय है। आपने अपनी मेहनत और ईमानदारी से हर एक चुनौती का सामना किया है। आपके योगदान ने न केवल टीम को मजबूती दी है, बल्कि हम सबको और भी बेहतर करने के लिए प्रेरित किया है।",
            "आपने जिस तरह से पिछले कुछ महीनों में कठिन परिस्थितियों को संभाला है, वह सराहनीय है। आपकी प्रेरणा और नेतृत्व से टीम ने बेहतरीन परिणाम हासिल किए हैं। आपका काम सभी के लिए प्रेरणा का स्रोत बन गया है।",
        ]
    }
}



def add_generated_examples(df, text_examples):
    new_data = []
    
    id_start = max(df['id'].max(), 0) + 1 if not df.empty else 1

    for label, languages in text_examples.items():
        for lang, texts in languages.items():
            for text in texts:
                row = {
                    "id": id_start,
                    "text": text,
                    "praise": 1 if label =="praise" else 0,
                    "demands": 1 if label == "demands" else 0,
                    "complaint": 1 if label =="complaint" else 0,
                    "questions": 1 if label == "questions" else 0
                }
                new_data.append(row)
                id_start += 1
                
    random.shuffle(new_data)
    return pd.concat([df, pd.DataFrame(new_data)], ignore_index=True)



df1 = add_generated_examples(df, text_examples)

# Display the updated dataframe
print(df1)


                 id                                               text  \
0      500796286320  Wow! From what I've observed from this documen...   
1      838906157157  काय रे dungnat मेंदु असणाऱ्या आंधभक्ता तुझा आई...   
2     1011026626743  अजित दादा आणि प्रफुल्ल पटेल यांनी केलेल्या काम...   
3     1068853499446  She's saying that "doing her own research" led...   
4      502772748919  That is not Karen, that is perfectly reasonabl...   
...             ...                                                ...   
4164  1099483387630  Classroom की condition देखी है तुमने? Fans तो ...   
4165  1099483387641  पाणीपुरवठा खंडित झालाय, सकाळपासून पाणी नाही. ल...   
4166  1099483387664  मुझे आपके द्वारा दी गई इंटरनेट सेवा से बहुत पर...   
4167  1099483387581  क्याआप बता सकते हैं कि मैंने पिछली बार जो कंप्...   
4168  1099483387644  The road near our colony is in such bad condit...   

      complaint  demands  praise  questions  
0           0.0      0.0     1.0        0.0  
1           1.0    

In [6]:
df1

,id,text,complaint,demands,praise,questions
0,500796286320,Wow! From what I've observed from this documen...,0.0,0.0,1.0,0.0
1,838906157157,काय रे dungnat मेंदु असणाऱ्या आंधभक्ता तुझा आई...,1.0,0.0,0.0,0.0
2,1011026626743,अजित दादा आणि प्रफुल्ल पटेल यांनी केलेल्या काम...,0.0,0.0,1.0,0.0
3,1068853499446,"She's saying that ""doing her own research"" led...",1.0,0.0,0.0,0.0
4,502772748919,"That is not Karen, that is perfectly reasonabl...",1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
4165,1099483387686,"आपका काम शानदार है! आपने जो भी किया, उसे न केव...",0.0,0.0,1.0,0.0
4166,1099483387671,The train services at the railway station are ...,1.0,0.0,0.0,0.0
4167,1099483387650,Library में books का arrangement इतना खराब है ...,1.0,0.0,0.0,0.0
4168,1099483387668,Tumhari food delivery service ne mujhe kaafi d...,1.0,0.0,0.0,0.0


In [6]:
stopwords = set(["the", "and", "to", "of", "a", "in", "for", "on", "with", "by", "an", "it", "from", "as", "be", "that","are","ka"])  

def remove_stopwords(text):
    text = text.lower()
    return " ".join([word for word in text.split() if word not in stopwords])

df1['text'] = df1['text'].apply(remove_stopwords)

In [7]:
df1

,id,text,complaint,demands,praise,questions
0,500796286320,wow! what i've observed this documentary natio...,0.0,0.0,1.0,0.0
1,838906157157,काय रे dungnat मेंदु असणाऱ्या आंधभक्ता तुझा आई...,1.0,0.0,0.0,0.0
2,1011026626743,अजित दादा आणि प्रफुल्ल पटेल यांनी केलेल्या काम...,0.0,0.0,1.0,0.0
3,1068853499446,"she's saying ""doing her own research"" led diff...",1.0,0.0,0.0,0.0
4,502772748919,"is not karen, is perfectly reasonable. she los...",1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
4166,1099483387687,"आपका काम शानदार है! आपने जो भी किया, उसे न केव...",0.0,0.0,1.0,0.0
4167,1099483387657,"gym equipment our society is poor condition, s...",1.0,0.0,0.0,0.0
4168,1099483387631,shazam bro i am your big fan if you see this m...,1.0,0.0,0.0,0.0
4169,1099483387554,"tujhe office supplies urgent chahiye kya , kal...",0.0,1.0,0.0,0.0


In [9]:
def get_language(text):
    return Detector("".join(x for x in text if x.isprintable()), quiet=True).languages[0].name

df1["language"] = df1["text"].progress_apply(get_language)
df
df2["language"]=df2["text"].progress_apply(get_language)

AttributeError: 'Series' object has no attribute 'progress_apply'

In [ ]:
df1

ENGLISH vs NON_ENGLISH

In [ ]:
lang_list = sorted(list(set(df1["language"])))
counts = [list(df1["language"]).count(cont) for cont in lang_list]
df = pd.DataFrame(np.transpose([lang_list, counts]))
df.columns = ["Languages", "Count"]
df["Count"] = df["Count"].apply(int)

df_en = pd.DataFrame(np.transpose([["English", "Non-English"], [max(counts), sum(counts) - max(counts)]]))
df_en.columns = ["Languages", "Count"]

fig = px.bar(df_en, x="Languages", y="Count", title="Language of comments", color="Languages", text="Count")
fig.update_layout(template="plotly_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.data[1].marker.line.color = 'rgb(0, 0, 0)'
fig.data[1].marker.line.width = 0.5
fig.data[0].textfont.color = "black"
fig.data[0].textposition = "outside"
fig.data[1].textfont.color = "black"
fig.data[1].textposition = "outside"
fig

In [ ]:
fig = px.bar(df.query("Languages != 'English' and Languages != 'un'").query("Count >= 0"),
             y="Languages", x="Count", title="Language of non-English comments", template="plotly_white", color="Languages", text="Count", orientation="h")
fig.update_traces(marker=dict(line=dict(width=0.75,
                                        color='black')),  textposition="outside")
fig.update_layout(showlegend=False)
fig

In [ ]:
lang_list = sorted(list(set(df2["language"])))
counts = [list(df2["language"]).count(cont) for cont in lang_list]
df = pd.DataFrame(np.transpose([lang_list, counts]))
df.columns = ["Languages", "Count"]
df["Count"] = df["Count"].apply(int)

df_en = pd.DataFrame(np.transpose([["English", "Non-English"], [max(counts), sum(counts) - max(counts)]]))
df_en.columns = ["Languages", "Count"]

fig = px.bar(df.query("Languages != 'English' and Languages != 'un'").query("Count >= 0"),
             y="Languages", x="Count", title="Language of non-English comments", template="plotly_white", color="Languages", text="Count", orientation="h")
fig.update_traces(marker=dict(line=dict(width=0.75,
                                        color='black')),  textposition="outside")
fig.update_layout(showlegend=False)
fig

**After looking through the data we can see that some texts listed has english language has some hinglish text containing hindi words written in english like kya bhadiya etc**

**looks like test set contains some data rows with languages that are not present in training set.... needs to handle this somehow**

In [ ]:
def new_len(x):
    if type(x) is str:
        return len(x.split())
    else:
        return 0

df1["comment_words"] = df1["text"].apply(new_len)
nums = df1.query("comment_words != 0 and comment_words < 200").sample(frac=0.1)["comment_words"]
fig = ff.create_distplot(hist_data=[nums],
                         group_labels=["All comments"],
                         colors=["coral"])

fig.update_layout(title_text="Comment words", xaxis_title="Comment words", template="simple_white", showlegend=False)
fig.show()

**Visualization of most occuring words in all the four categories of classfication**

In [ ]:
# Function to get all text for a specific category
def get_text_by_category(df, category_column):
    """
    Combine all the text data corresponding to a specific binary category column.
    """
    texts = df[df[category_column] == 1]['text'].values
    return " ".join(texts)

# Function to get frequent words
def get_frequent_words(text, top_n=20):
    """
    Get the top N frequent words from the text.
    """
    words = text.split()
    word_counts = Counter(words)
    return word_counts.most_common(top_n)

def generate_word_cloud(text, category_name):
    """
    Generate and display a word cloud for a given category.
    """
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Word Cloud for {category_name}", fontsize=16)
    plt.show()
    
def plot_top_words(text, category_name, top_n=20):
    """
    Plot a bar chart of the most frequent words for a given category.
    """
    frequent_words = get_frequent_words(text, top_n=top_n)
    words, counts = zip(*frequent_words)
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(counts), y=list(words), palette="viridis")
    plt.title(f"Top {top_n} Words in {category_name}", fontsize=16)
    plt.xlabel("Frequency")
    plt.ylabel("Words")
    plt.show()



In [ ]:
# List of category columns
category_columns = ['complaint','demands','praise','questions']

for category in category_columns:
    print(f"Processing category: {category}")
    
    # Get text for this category
    category_text = get_text_by_category(df1, category)
    
    # Generate Word Cloud
    generate_word_cloud(category_text, category)
    
    # Plot Top Words
    plot_top_words(category_text, category, top_n=20)


In [ ]:
def get_text_by_language_and_category(df, category, language):
    """
    Get all text data for a specific category and language.
    """
    texts = df[(df[category] == 1) & (df['language'] == language)]['text'].values
    return " ".join(texts)

# Example for generating word clouds for all languages
languagess = ['Urdu', 'Malayalam', 'Sinhala',
       'Uzbek', 'Afar', 'Persian', 'Czech', 'Southern Sotho', 'un',
       'Tagalog', 'Indonesian', 'Manx', 'Akan', 'Tamil',

              'Scottish Gaelic']  # Add detected languages here

for category in category_columns:
    for language in languagess:
        print(f"Category: {category}, Language: {language}")
        
        # Get text for this language and category
        language_category_text = get_text_by_language_and_category(df1, category, language)
        
        if language_category_text:  # Check if there is text for this combination
            # Generate Word Cloud
            generate_word_cloud(language_category_text, f"{category} ({language})")
            
            # Plot Top Words
            plot_top_words(language_category_text, f"{category} ({language})", top_n=20)


In [ ]:
labels_remove=['Uzbek', 'Afar', 'Czech', 'Southern Sotho','Urdu','Tamil','Akan','Persian','Scottish Gaelic','Manx']

In [ ]:
df1_filtered = df1[~df1['language'].isin(labels_remove)]
df1_filtered['language'].unique()

In [ ]:
!pip install indic-nlp-library


In [ ]:
import string
from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator
from indicnlp.tokenize import indic_tokenize


def process_english_text(text):
    """
    Preprocess English text: remove stopwords, lowercase, and strip punctuation.
    """
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower()) 
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return " ".join(tokens)


#def process_hinglish_text(text):
    """
    Preprocess Hinglish text by transliterating and cleaning it.
    """
   # try:
        #transliterated = UnicodeIndicTransliterator.transliterate(text, 'h', 'en')
       # tokens = word_tokenize(transliterated.lower())
      #  tokens = [word for word in tokens if word not in string.punctuation]
     #   return " ".join(tokens)
   # except Exception as e:
    #    return text.lower()

def process_hindi_text(text):
    """
    Preprocess Hindi text: tokenize and remove punctuations.
    """
    try:
        tokens = list(indic_tokenize.trivial_tokenize(text))
        tokens = [word for word in tokens if word not in string.punctuation]
        return " ".join(tokens)
    except Exception as e:
        return text
        
def process_marathi_text(text):
    """
    Preprocess Marathi text: tokenize and remove punctuations.
    """
    try:
        tokens = list(indic_tokenize.trivial_tokenize(text))
        tokens = [word for word in tokens if word not in string.punctuation]
        return " ".join(tokens)
    except Exception as e:
        return text
def process_minor_language_text(text):
    """
    Preprocess text in minority languages by removing punctuation and normalizing.
    """
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word not in string.punctuation]
    return " ".join(tokens)



In [ ]:
def process_text_by_language(row):
    """
    Process text based on the detected language.
    """
    lang = row['language']
    text = row['text']

    if lang == 'English':
        return process_english_text(text)
    #elif lang == 'Hinglish':
        #return process_hinglish_text(text)
    elif lang == 'Hindi':
        return process_hindi_text(text)
    elif lang == 'Marathi':
        return process_marathi_text(text)
    elif lang in ['Urdu', 'Malayalam', 'Sinhala',
       'Uzbek', 'Afar', 'Persian', 'Czech', 'Southern Sotho', 'un',
       'Tagalog', 'Indonesian', 'Manx', 'Akan', 'Tamil',
       'Scottish Gaelic']:
        return process_minor_language_text(text)
    else:
        return text  


In [ ]:
import nltk


nltk.download('stopwords')


In [ ]:
df1['text'] = df1.apply(process_text_by_language, axis=1)
df2['text']=df2.apply(process_text_by_language,axis=1)


In [ ]:
df1

**Tokenization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=4000)
X = tfidf_vectorizer.fit_transform(df1['text'])
Xt = tfidf_vectorizer.fit_transform(df2['text'])


In [ ]:
Xt

**Embedding**

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  
X = embedder.encode(df1['text'].tolist())
Xt = embedder.encode(df2['text'].tolist())


In [ ]:
Xt

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


model_name = "xlm-roberta-base"  # or "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval() 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
import numpy as np

def batch_embed(texts, tokenizer, model, batch_size=16, max_length=128):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        tokens = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        tokens = {key: value.to(device) for key, value in tokens.items()}
        with torch.no_grad():
            outputs = model(**tokens)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

In [ ]:

train_texts = df1["text"].tolist()
test_texts = df2["text"].tolist()


In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:

X_train_features = batch_embed(train_texts, tokenizer, model)
X_test_features = batch_embed(test_texts, tokenizer, model)
X_train_features


In [ ]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X_train_features, df1[['praise', 'complaint', 'demands', 'questions']], test_size=0.2, random_state=42)

In [ ]:
y1_test

In [ ]:
def custom_loss_function(y_true, y_pred):
    """
    Custom loss function for multilabel classification with XGBoost.
    We combine Binary Cross-Entropy (BCE) with an entropy regularization term.
    """
    # Sigmoid transformation to convert logits to probabilities
    preds = 1.0 / (1.0 + np.exp(-y_pred))  # Apply sigmoid function
    
    # BCE loss: binary cross-entropy for each label
    bce_loss = -y_true * np.log(preds + 1e-15) - (1 - y_true) * np.log(1 - preds + 1e-15)
    bce_loss = np.sum(bce_loss, axis=1)  # Sum over all labels
    
    # Regularization term: Encourage diversity in predictions
    # This is the entropy of the predicted probabilities for each instance
    entropy_loss = -np.sum(preds * np.log(preds + 1e-15), axis=1)  # Entropy across labels
    
    # Total loss: BCE loss + entropy regularization
    total_loss = np.mean(bce_loss + 0.1 * entropy_loss)  # 0.1 is a weight for regularization
    
    # Compute gradients and hessians (used by XGBoost)
    grad = preds - y_true
    hess = preds * (1 - preds)  # Sigmoid derivative
    
    return grad, hess

In [ ]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

# Prepare storage for predictions and scores
y_pred = {}
#auc_scores = {}

# Train and evaluate for each label independently
for col in ['complaint','demands', 'praise','questions']:
    # Prepare data for the current label
    dtrain = xgb.DMatrix(X_train_features, label=df1[col])
    dtest = xgb.DMatrix(X_test_features)
    
    # Define XGBoost parameters
    params = {
        'objective':custom_loss_function,  # Binary classification
        'eval_metric': 'auc',           # AUC for evaluation
        'learning_rate': 0.1,
        'max_depth': 3,
        'subsample': 0.8,
        'lambda': 1,
        'alpha': 1
    }
    
    # Train the model
    xgb_model= xgb.train(params, dtrain, num_boost_round=500)
    
    for col in ['complaint', 'demands', 'praise', 'questions']:
        
        dtest = xgb.DMatrix(X_test_features) 
        y_pred[col] =xgb_model.predict(dtest)
       # preds = xgb_model.predict(dtest)
        
    
    
    #auc_scores[col] = roc_auc_score(y1_test[col], preds)

    #print("AUC Scores:", auc_scores)
    output_df = pd.DataFrame({
    'id': df2['id'],
    'complaint': y_pred['complaint'],
    'demands': y_pred['demands'],
    'praise': y_pred['praise'],
    'questions': y_pred['questions']
})

# Save the predictions to a CSV file
output_df.to_csv('prediction2.csv', index=False)





In [ ]:
y_pred

In [ ]:
output_df

In [ ]:
y_pred

In [ ]:
y_predd = {}

# Process and predict for each label
for col in ['complaint', 'demands', 'praise', 'questions']:
    # Prepare the test data as XGBoost DMatrix
    model1 = xgb.Booster()
    model1.load_model(f"/kaggle/working/{col}_xgb_model.json")
    dtestt = xgb.DMatrix(X_test_features) 
    y_predd[col] =model1.predict(dtestt)

# Combine predictions into a DataFrame
output_df = pd.DataFrame({
    'id': df2['id'],
    'complaint': y_predd['complaint'],
    'demands': y_predd['demands'],
    'praise': y_predd['praise'],
    'questions': y_predd['questions']
})

# Save the predictions to a CSV file
output_df.to_csv('prediction2.csv', index=False)




In [ ]:
model1.get_dump()

In [ ]:
d=pd.read_csv("/kaggle/working/predictions.csv")
d

In [ ]:
y_predd

In [ ]:
pip install transformers datasets torch scikit-learn


In [ ]:
from transformers import DistilBertTokenizer
from sklearn.model_selection import train_test_split
import torch


tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')


def tokenize_data(texts, max_length=128):
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )


#X_train, X_test, y_train, y_test = train_test_split(
 #   df1['text'], df1[['complaint', 'demands', 'praise', 'questions']],
  #  test_size=0.2,
   # random_state=42
#)


train_tokens = tokenize_data(df1['text'])
test_tokens = tokenize_data(df1['text'])

y_train_tensor = torch.tensor(df1[['complaint', 'demands', 'praise', 'questions']].values, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
#y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)


In [ ]:
from transformers import DistilBertModel
import torch.nn as nn

class DistilBERTMultiLabel(nn.Module):
    def __init__(self, num_labels):
        super(DistilBERTMultiLabel, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
        self.classifier = nn.Sequential(
            nn.Linear(self.distilbert.config.hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_labels),
            nn.Sigmoid()  
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0, :]  # CLS token representation
        logits = self.classifier(hidden_state)
        return logits


model = DistilBERTMultiLabel(num_labels=4)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from torch.optim import lr_scheduler

# Prepare data loaders
train_data = TensorDataset(
    train_tokens['input_ids'], train_tokens['attention_mask'], y_train_tensor
)
test_data = TensorDataset(
    test_tokens['input_ids'], test_tokens['attention_mask']
)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5,weight_decay=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.2)

# Loss function
criterion = nn.BCELoss()  # Binary cross-entropy for multi-label
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


In [ ]:
print(f"Input IDs size: {train_tokens['input_ids'].size()}")
print(f"Attention Mask size: {train_tokens['attention_mask'].size()}")
print(f"Labels size: {y_train_tensor.size()}")


In [ ]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [ ]:
from tqdm import tqdm

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Training Loss: {train_loss / len(train_loader)}")
    scheduler.step()


In [ ]:
d=pd.read_csv("/kaggle/working/submission3.csv")
d

In [ ]:
from sklearn.metrics import roc_auc_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids, attention_mask)
        all_preds.append(outputs.cpu())
        all_labels.append(labels.cpu())

all_preds = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()

auc_scores = {}
for i, col in enumerate(['complaint', 'demands', 'praise', 'questions']):
    auc_scores[col] = roc_auc_score(all_labels[:, i], all_preds[:, i])

print("AUC Scores:", auc_scores)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Prepare test dataset and dataloader
test_dataset = TensorDataset(test_tokens['input_ids'], test_tokens['attention_mask'])
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)  # Use a small batch size

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Predict in batches
test_preds = []

model.eval()  # Set model to evaluation mode
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = [x.to(device) for x in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        test_preds.append(outputs.cpu().numpy())

# Concatenate predictions from all batches
test_preds = np.vstack(test_preds)  # Convert list of arrays to a single array

# Create submission file
submission = pd.DataFrame({
    'id': df2['id'],
    'complaint': test_preds[:, 0],
    'demands': test_preds[:, 1],
    'praise': test_preds[:, 2],
    'questions': test_preds[:, 3]
})

submission.to_csv('submission4.csv', index=False)


In [ ]:
# Predict on test data
#test_token = tokenize_data(df2['text'])
test_inputs = {
    'input_ids': test_tokens['input_ids'].to(device),
    'attention_mask': test_tokens['attention_mask'].to(device)
}

# Move model to device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
with torch.no_grad():
    test_preds = model(**test_inputs).cpu().numpy()

# Create submission file
submission = pd.DataFrame({
    'id': df2['id'],
    'complaint': test_preds[:, 0],
    'demands': test_preds[:, 1],
    'praise': test_preds[:, 2],
    'questions': test_preds[:, 3]
})

submission.to_csv('submission4.csv', index=False)


In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [ ]:
dd=pd.read_csv("/kaggle/working/submission.csv")
dd

In [ ]:
from transformers import XLMRobertaTokenizer, BertTokenizer

# Load tokenizers
xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
#mbert_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def preprocess_text(texts, tokenizer, max_len=128):
    encodings = tokenizer.batch_encode_plus(
        texts.tolist(),
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    return encodings

xlm_encodings = preprocess_text(df1['text'], xlm_tokenizer)
#mbert_encodings = preprocess_text(df1['text'], mbert_tokenizer)

In [ ]:
import torch.nn as nn
from transformers import AutoModel



class TransformerClassifier(nn.Module):
    def __init__(self, model_name, num_labels):
        super(TransformerClassifier, self).__init__()
        self.transformer = AutoModel.from_pretrained(model_name)
        
        # Classifier head
        self.classifier = nn.Sequential(
            nn.Linear(self.transformer.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, num_labels),
            nn.Sigmoid()  # Normalize probabilities across all labels
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0, :]  # CLS token
        logits = self.classifier(hidden_state)
        return logits


# Initialize models
xlm_model = TransformerClassifier("xlm-roberta-base", num_labels=4)
#mbert_model = TransformerClassifier("bert-base-multilingual-cased", num_labels=4)



In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

# Define targets
y = df1[['complaint', 'demands', 'praise', 'questions']].values

# Prepare datasets and loaders
train_dataset = TextDataset(xlm_encodings, y)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)


In [ ]:
from sklearn.metrics import roc_auc_score
from transformers import AdamW
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, train_loader, epochs=5, lr=1e-3):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = AdamW(model.parameters(), lr=lr,weight_decay=0.01)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        all_labels, all_preds = [], []
        
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            all_labels.append(labels.cpu().numpy())
            all_preds.append(torch.sigmoid(outputs).detach().cpu().numpy())
        
        # Calculate AUC-ROC
        all_labels = np.vstack(all_labels)
        all_preds = np.vstack(all_preds)
        auc_scores = {col: roc_auc_score(all_labels[:, i], all_preds[:, i]) 
                      for i, col in enumerate(['complaint', 'demands', 'praise', 'questions'])}
        
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, AUC-ROC: {auc_scores}")

# Train XLM-RoBERTa
train_model(xlm_model, train_loader)

# Train mBERT
#train_model(mbert_model, train_loader)


In [ ]:
# Collect predictions for meta-classifier
xlm_preds = xlm_model(xlm_encodings['input_ids'].to(device), xlm_encodings['attention_mask'].to(device)).detach().cpu().numpy()
#mbert_preds = mbert_model(mbert_encodings['input_ids'].to(device), mbert_encodings['attention_mask'].to(device)).detach().cpu().numpy()

#meta_features = np.hstack([xlm_preds, mbert_preds])
#meta_clf = LogisticRegression()
#meta_clf.fit(meta_features, y)


In [ ]:
#test_df = pd.read_csv("test.csv")
test_encodings = preprocess_text(df2['text'], xlm_tokenizer)


In [ ]:
xlm_test_preds = xlm_model(test_encodings['input_ids'].to(device), test_encodings['attention_mask'].to(device)).detach().cpu().numpy()
#mbert_test_preds = mbert_model(test_encodings['input_ids'].to(device), test_encodings['attention_mask'].to(device)).detach().cpu().numpy()

# Combine predictions
#test_meta_features = np.hstack([xlm_test_preds, mbert_test_preds])
#final_preds = meta_clf.predict_proba(test_meta_features)


In [ ]:
submission = pd.DataFrame({
    "id": df2["id"],
    "complaint": final_preds[:, 0],
    "demands": final_preds[:, 1],
    "praise": final_preds[:, 2],
    "questions": final_preds[:, 3]
})
submission.to_csv("submission.csv", index=False)


In [4]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


In [5]:
train_texts = df1['text']  
train_labels = df1[['complaint', 'demands', 'praise', 'questions']].values

test_texts = df2['text']  

In [6]:
from transformers import AutoTokenizer
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")


# Tokenize data
def tokenize_texts(texts, max_length=128):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

train_encodings = tokenize_texts(train_texts)

test_encodings = tokenize_texts(test_texts)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Dataset class
class MultilabelDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

# Create datasets
train_dataset = MultilabelDataset(train_encodings, train_labels)
test_dataset = MultilabelDataset(test_encodings)


In [8]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [9]:
from torch.nn import BCEWithLogitsLoss

def train_model(model, dataloader, optimizer, epochs=9):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            inputs = {key: val.to("cuda") for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to("cuda")

            optimizer.zero_grad()
            outputs = model(**inputs).logits
            loss = BCEWithLogitsLoss()(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")


In [10]:
from transformers import AutoModelForSequenceClassification
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-large", num_labels=4, problem_type="multi_label_classification").to("cuda")




config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import get_linear_schedule_with_warmup
# Optimizer
optimizer = AdamW(model.parameters(), lr=1.5e-5)

# Add warmup and decay
#num_training_steps = len(train_loader) * 6
#scheduler = get_linear_schedule_with_warmup(
 #   optimizer,
  #  num_warmup_steps=num_training_steps * 0.05,  # 10% warmup
   # num_training_steps=num_training_steps
#)


# Train the model
train_model(model, train_loader, optimizer, epochs=8)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-7-2acccb9edd82>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/8, Loss: 0.4022
Epoch 2/8, Loss: 0.2412
Epoch 3/8, Loss: 0.1707
Epoch 4/8, Loss: 0.1218
Epoch 5/8, Loss: 0.0940
Epoch 6/8, Loss: 0.0679
Epoch 7/8, Loss: 0.0495
Epoch 8/8, Loss: 0.0392


In [12]:
def predict(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = {key: val.to("cuda") for key, val in batch.items()}
            outputs = model(**inputs).logits
            preds = torch.sigmoid(outputs).cpu().numpy()
            predictions.extend(preds)
    return np.array(predictions)

# Get predictions
test_predictions = predict(model, test_loader)

# Normalize predictions to sum to 1
normalized_predictions = test_predictions / test_predictions.sum(axis=1, keepdims=True)


<ipython-input-7-2acccb9edd82>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [13]:
# Create a DataFrame for submission
submission = pd.DataFrame({
    'id': df2['id'],  # Add the test IDs
    'complaint': normalized_predictions[:, 0],
    'demands': normalized_predictions[:, 1],
    'praise': normalized_predictions[:, 2],
    'questions': normalized_predictions[:, 3],
})

# Save to CSV
submission.to_csv('submission11.csv', index=False)
print("Submission file saved as submission.csv")


Submission file saved as submission.csv


In [44]:
d=pd.read_csv("/kaggle/working/submission4.csv")
d

,id,complaint,demands,praise,questions
0,1041016773991,0.001530,0.990229,0.006878,0.001363
1,109362481297,0.000938,0.001041,0.997162,0.000859
2,985019053532,0.996231,0.001143,0.001784,0.000842
3,436629695381,0.996317,0.001138,0.000639,0.001907
4,585196067684,0.002049,0.991497,0.002880,0.003574
...,...,...,...,...,...
1995,148770317688,0.996688,0.001272,0.001087,0.000953
1996,288582831883,0.000644,0.002317,0.996471,0.000568
1997,285086247879,0.000942,0.000979,0.997126,0.000953
1998,114758927004,0.006517,0.002385,0.002276,0.988822


In [14]:
submission

,id,complaint,demands,praise,questions
0,1041016773991,0.006667,0.985395,0.004397,0.003541
1,109362481297,0.002438,0.001627,0.995204,0.000731
2,985019053532,0.903841,0.001412,0.094509,0.000238
3,436629695381,0.991679,0.000561,0.006366,0.001394
4,585196067684,0.004007,0.983747,0.005093,0.007153
...,...,...,...,...,...
1995,148770317688,0.990831,0.000781,0.007598,0.000790
1996,288582831883,0.002646,0.003159,0.993678,0.000517
1997,285086247879,0.002626,0.001765,0.994819,0.000790
1998,114758927004,0.971603,0.000917,0.006551,0.020929


In [28]:
ddd=pd.read_csv("/kaggle/input/submission1-5/submission1.5.csv")
ddd

,id,complaint,demands,praise,questions
0,1041016773991,0.028546,0.920891,0.032976,0.017587
1,109362481297,0.007149,0.008302,0.975564,0.008985
2,985019053532,0.950405,0.011815,0.035136,0.002644
3,436629695381,0.968888,0.011756,0.011268,0.008087
4,585196067684,0.030264,0.913229,0.029869,0.026638
...,...,...,...,...,...
1995,148770317688,0.966109,0.013035,0.015271,0.005585
1996,288582831883,0.006159,0.782621,0.202552,0.008667
1997,285086247879,0.014307,0.007581,0.974061,0.004051
1998,114758927004,0.175441,0.007197,0.012809,0.804553


In [36]:
d4=pd.read_csv('/kaggle/working/submission4.csv')
d4

,id,complaint,demands,praise,questions
0,1041016773991,0.001530,0.990229,0.006878,0.001363
1,109362481297,0.000938,0.001041,0.997162,0.000859
2,985019053532,0.996231,0.001143,0.001784,0.000842
3,436629695381,0.996317,0.001138,0.000639,0.001907
4,585196067684,0.002049,0.991497,0.002880,0.003574
...,...,...,...,...,...
1995,148770317688,0.996688,0.001272,0.001087,0.000953
1996,288582831883,0.000644,0.002317,0.996471,0.000568
1997,285086247879,0.000942,0.000979,0.997126,0.000953
1998,114758927004,0.006517,0.002385,0.002276,0.988822


In [23]:
d

NameError: name 'd' is not defined